<a href="https://colab.research.google.com/github/didi64/Colab_Test/blob/main/Classify_Reviews2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
ROOT = '/content/drive/My Drive/CAS_Gregi/'
drive.mount('/content/drive', force_remount=True)

nb_path = '/content/notebooks'
os.symlink(ROOT + 'CO_modules', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [3]:
import pickle
with open(ROOT + 'data/hotelreviews_cleaned_train_test_spell.pkl','rb') as f:
    data_train, data_test = pickle.load(f)

In [4]:
import random
import numpy as np
import tensorflow as tf
from functools import reduce

In [5]:
def fix_shape(text):
    return tf.constant(text, shape = (1,))

def get_dataset(d):
    ''' get dataset from dict {score: list_of_revies,...}'''
    # list of tuples [(score, review),...]
    all_reviews = reduce(lambda x,y: x + y, [[(r, k) for k,v in d.items() for r in v]])
    random.shuffle(all_reviews)
    reviews = tf.constant([r for r,_ in all_reviews], shape=(len(all_reviews), 1))
    gpt_scores  = tf.constant([s - 1 for _, s in all_reviews])

    n = len(reviews)
    ds = tf.data.Dataset.from_tensor_slices((reviews[:n], gpt_scores[:n]))
    return ds

In [6]:
ds_train = get_dataset(data_train)
ds_test = get_dataset(data_test)

In [11]:
all_reviews = reduce(lambda x,y: x + y, [[(r, k) for d in (data_train, data_test) for k,v in d.items() for r in v]])
WORDS = reduce(lambda x,y:x|y, [set(x[0].split()) for x in all_reviews])

In [12]:
len(WORDS)

9033

In [13]:
# standardization: one element of  standardizations or a callable mapping strings to strings
standardizations = [None, "lower_and_strip_punctuation", "lower", "strip_punctuation"]
max_tokens = 10_000
ngrams = None
sequence_length = 200

tv_layer = tf.keras.layers.TextVectorization(
    max_tokens = max_tokens,
    standardize = standardizations[0],
    ngrams=ngrams,
    output_mode='int',
    output_sequence_length = sequence_length,
    pad_to_max_tokens=True,
)

In [14]:
tv_layer.adapt([r[0] for r in all_reviews])
vocab = tv_layer.get_vocabulary()
len(vocab) # max_tokens

9035

In [15]:
tv_layer(data_train[3][30])

<tf.Tensor: shape=(200,), dtype=int64, numpy=
array([  16,    8,  436,   82,  163,    5,  365,  222,   37,   21,    2,
        377,  259,   36,   20,    5,  276,   10,  104, 2640,   21,    2,
        377,  466,   16,   20,  528,    2,  207,   55,    5,  118,  275,
          3,   44,  105,  210,  192, 1695,    6,  316,    8,    2,  115,
          7,   13,    2,   94,   25,   65,  264,   42,  367,    3,  586,
          5,  664,    4,  392,    8,    2,    7,   36,   14,   40, 1155,
        107,  316,   17,  429,   29,  296,    5,  531,    3,    5,  357,
        113,   28,   89,   28,  291,  398,    6, 4099,    5, 1109, 3744,
          2,    7,    4,   52,    3,  722,    3,    2,   41,    4,   98,
         30,  106,  544,    8,    2,   67,    3,  738,  585, 1656,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
     

In [16]:
output_dim =  16
em_layer = tf.keras.layers.Embedding(input_dim = max_tokens + 1, output_dim= output_dim)
em_layer(tv_layer('what a hotel'))

<tf.Tensor: shape=(200, 16), dtype=float32, numpy=
array([[ 0.03915811,  0.04589141, -0.02808626, ...,  0.03733112,
        -0.0449727 ,  0.04960599],
       [-0.01326507,  0.01720462,  0.04984209, ...,  0.02812301,
        -0.00649334, -0.00091988],
       [ 0.03496058,  0.01939512,  0.02389786, ..., -0.01333567,
        -0.0126762 , -0.01191963],
       ...,
       [-0.01344358,  0.00480167, -0.03905276, ..., -0.03875424,
         0.01712754,  0.04231875],
       [-0.01344358,  0.00480167, -0.03905276, ..., -0.03875424,
         0.01712754,  0.04231875],
       [-0.01344358,  0.00480167, -0.03905276, ..., -0.03875424,
         0.01712754,  0.04231875]], dtype=float32)>

In [18]:
model = tf.keras.Sequential([
    tv_layer,
    em_layer,
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(10),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'relu')])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 200)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 200, 16)           160016    
                                                                 
 dropout (Dropout)           (None, 200, 16)           0         
                                                                 
 lstm (LSTM)                 (None, 10)                1080      
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 flatten (Flatten)           (None, 10)                0         
                                                        

In [20]:
metric = tf.keras.metrics.MeanAbsoluteError(name='mean_absolute_error', dtype=None)
loss =  tf.keras.losses.MeanSquaredError()
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=loss,
    metrics=[metric],
)

In [21]:
# batch train and validate date if this not already happend
if next(iter(ds_train))[0].shape[0] == 1:
    ds_train = ds_train.batch(32)

if next(iter(ds_test))[0].shape[0] == 1:
    ds_test = ds_test.batch(32)

In [22]:
i = 0
for review, _ in iter(ds_train):
    i += 1
    if (i %10) ==0:
        print(i, end = ', ')
    model(review)

10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 

In [ ]:
model.fit(
    ds_train,
    epochs = 5,
    validation_data = ds_test,
)

Epoch 1/5
157/157 [==============================] - 17s 92ms/step - loss: 3.0928 - mean_absolute_error: 1.4469 - val_loss: 2.0202 - val_mean_absolute_error: 1.2280
Epoch 2/5
 39/157 [======>.......................] - ETA: 8s - loss: 2.2806 - mean_absolute_error: 1.2881

In [ ]:
model.evaluate(ds_test)

In [ ]:
# save the model
model.save(ROOT + 'data/models/lstm10.sav')

In [ ]:
text = fix_shape('the best hotel i ever stayed at')
model.predict(text)

In [ ]:
text = '''\
the breakfast was excellent and the room very clean and spacious.
the staff was very friendly.
I can really recommend this place'''
model.predict(fix_shape(text))

In [ ]:
text = '''\
the breakfast was excellent and the room very clean and spacious.
the staff was very friendly.
I can relly recommend this place'''
model.predict(fix_shape(text))

In [ ]:
text = '''\
the breakfast was ok, but nothing special.
the staff was very friendly.
But the room we small and not suitable for working'''
model.predict(fix_shape(text))

In [ ]:
text = '''\
the breakfast was ok, but nothing special.
the staff was quite rude.
And the room we small and not suitable for working'''
model.predict(fix_shape(text))

In [ ]:
text = '''\
the breakfast was bad, only bread and coffe.
Checkin took forever.
No roomservice.
The room we small and not suitable for working'''
model.predict(fix_shape(text))